# Ejercicios Pair Programming - Módulo 2 Sprint 2
## Lección 14 - ETL II

Tendréis que usar el csv attacks_limpieza_completa que tenéis adjunto abajo.

En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:

El csv con los ataques de tiburones que hemos estado limpiando hasta ahora, el que os hemos adjuntado (attacks_limpieza_completa). Sentiros libres de usar vuestros propios csv en caso de que queráis.

El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer.

El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:

Cargaremos los dos ficheros de datos

Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:

USA

Australia

New Zealand

South Africa

Papua New Guinea

Del dataframe de los datos climáticos seleccionaremos todas las columnas.

Cuando ya tengamos todos los datos deseados juntaremos los dos csv.

Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.

Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. 

⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

Os recomendamos resetear el index del dataframe de los datos climáticos para que no se repitan los nombres de las columnas.
​
​
El primer problema al que nos podemos enfrentar es que si vemos los tipos de las columnas vemos que estas columnas son objetos, es decir, strings, lo que hará que trabajar con ellas sea un poco complicado: 

clima.dtypes
​
timepoint             int64

cloudcover            int64

highcloud             int64

midcloud              int64

lowcloud              int64

rh_profile           object

wind_profile         object
​
En Python tenemos la librería `ast` que nos permite castear un string que dentro tiene diccionarios, o listas o tuplas a su tipo correspondiente. En nuestro caso, lo que conseguiremos es no tener strings sino listas en la columna. Esto lo haremos de la siguiente forma: 
​
import ast
​
clima['wind_profile']= clima['wind_profile'].apply(ast.literal_eval)
​
Una vez que tengamos la columna cambiada, una fantasía de Pandas es que si hago un apply sobre una columna cuyos valores son diccionarios o listas nos va a genererar una columna con los valores de los diccionarios o listas. Donde cada columna será key del diccionario o cada elemento de la lista. 
​
​
x = clima['wind_profile'].apply(pd.Series)
​
​
Nos creamos un dataframe nuevo con el resultado de la información de una de las columnas separadas por columnas. Esto nos va a devolver un dataframe donde cada fila será una celda del dataframe anterior. 

x = df['rh_profile'].apply(pd.Series) 
​
¿Qué es lo que ocurre cuando hacemos esto?

Nos ha creado tantas columnas como valores tuvieramos en la lista. Donde columna es, en este caso, un diccionario (porque nuestra lista esta compuesta por distintos diccionarios)
​
Ok, hemos conseguido desempaquetar la información de la lista en distintas columnas. Ahora tenemos que despempaquetar la información de los diccionarios en distintas columnas. En este caso, lo que querremos es que las key de los diccionarios sean los nombres de las columnas y los values los valores de las celdas del dataframe. Volveremos a seguir entonces la misma lógica que antes con el apply, pero en este caso necesitamos hacerlo para todo el dataframe (que es x): 
​
Por eso empezamos con un for para iterar por cada una de las columnas. 

for i in range(len(x.columns)): 
​
    aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(x[i].apply(pd.Series)["layer"][0]) 
​
    hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(x[i].apply(pd.Series)["rh"] )
​
    usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df.insert(i, nombre, valores)
​
Una vez que hayamos hecho esto para las dos columnas ya podremos hacer el gropuby para después unir toda la información. 

Guardar los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.

Happy coding 🦈

In [1]:
import pandas as pd
import requests
import ast
import numpy as np

In [2]:
pd.options.display.max_columns=None

In [3]:
## Cargamos los dos ficheros
df_ataques = pd.read_csv("attacks_limpieza_completa.csv", index_col=0)
df_clima = pd.read_csv("etl1.csv", index_col=0)

In [4]:
df_ataques.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860


In [5]:
df_clima.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 14}, {'layer': '900m...","[{'layer': '950mb', 'direction': 95, 'speed': ...",26,-1,12,1011,none,1,0,95,3,papua new guinea
1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 90, 'speed': ...",26,-1,12,1013,none,1,0,90,3,papua new guinea


In [6]:
df_clima.shape

(320, 17)

In [7]:
# Creamos un filtro para quedarnos con los paises seleccionados anteriormente en el dataframe clima
df_ataques = df_ataques[df_ataques["country"].isin(["usa","australia","new zealand", "south africa", "papua new guinea"])]

In [8]:
df_ataques["country"].unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

### rh_profile

In [9]:
df_clima.reset_index(inplace=True)

In [10]:
df_clima["rh_profile"].dtypes

dtype('O')

In [11]:
df_clima['rh_profile']= df_clima['rh_profile'].apply(ast.literal_eval)

In [12]:
df_rh = df_clima["rh_profile"].apply(pd.Series)
df_rh.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': 14}","{'layer': '900mb', 'rh': 12}","{'layer': '850mb', 'rh': 7}","{'layer': '800mb', 'rh': 5}","{'layer': '750mb', 'rh': 5}","{'layer': '700mb', 'rh': 2}","{'layer': '650mb', 'rh': -1}","{'layer': '600mb', 'rh': -2}","{'layer': '550mb', 'rh': -3}","{'layer': '500mb', 'rh': -3}","{'layer': '450mb', 'rh': -3}","{'layer': '400mb', 'rh': 0}","{'layer': '350mb', 'rh': -1}","{'layer': '300mb', 'rh': 2}","{'layer': '250mb', 'rh': 2}","{'layer': '200mb', 'rh': -1}"
1,"{'layer': '950mb', 'rh': 12}","{'layer': '900mb', 'rh': 8}","{'layer': '850mb', 'rh': 6}","{'layer': '800mb', 'rh': 6}","{'layer': '750mb', 'rh': 3}","{'layer': '700mb', 'rh': -1}","{'layer': '650mb', 'rh': -3}","{'layer': '600mb', 'rh': -2}","{'layer': '550mb', 'rh': -3}","{'layer': '500mb', 'rh': -3}","{'layer': '450mb', 'rh': -3}","{'layer': '400mb', 'rh': -2}","{'layer': '350mb', 'rh': 0}","{'layer': '300mb', 'rh': 2}","{'layer': '250mb', 'rh': 0}","{'layer': '200mb', 'rh': -1}"
2,"{'layer': '950mb', 'rh': 14}","{'layer': '900mb', 'rh': 8}","{'layer': '850mb', 'rh': 5}","{'layer': '800mb', 'rh': 6}","{'layer': '750mb', 'rh': 6}","{'layer': '700mb', 'rh': 1}","{'layer': '650mb', 'rh': -3}","{'layer': '600mb', 'rh': -2}","{'layer': '550mb', 'rh': -3}","{'layer': '500mb', 'rh': -3}","{'layer': '450mb', 'rh': -4}","{'layer': '400mb', 'rh': -1}","{'layer': '350mb', 'rh': 1}","{'layer': '300mb', 'rh': 0}","{'layer': '250mb', 'rh': 1}","{'layer': '200mb', 'rh': -2}"
3,"{'layer': '950mb', 'rh': 14}","{'layer': '900mb', 'rh': 10}","{'layer': '850mb', 'rh': 8}","{'layer': '800mb', 'rh': 6}","{'layer': '750mb', 'rh': 4}","{'layer': '700mb', 'rh': 3}","{'layer': '650mb', 'rh': -2}","{'layer': '600mb', 'rh': -2}","{'layer': '550mb', 'rh': -2}","{'layer': '500mb', 'rh': -3}","{'layer': '450mb', 'rh': -3}","{'layer': '400mb', 'rh': 2}","{'layer': '350mb', 'rh': -1}","{'layer': '300mb', 'rh': 1}","{'layer': '250mb', 'rh': 0}","{'layer': '200mb', 'rh': 0}"
4,"{'layer': '950mb', 'rh': 15}","{'layer': '900mb', 'rh': 12}","{'layer': '850mb', 'rh': 10}","{'layer': '800mb', 'rh': 6}","{'layer': '750mb', 'rh': 5}","{'layer': '700mb', 'rh': 2}","{'layer': '650mb', 'rh': -2}","{'layer': '600mb', 'rh': -2}","{'layer': '550mb', 'rh': -2}","{'layer': '500mb', 'rh': -2}","{'layer': '450mb', 'rh': -1}","{'layer': '400mb', 'rh': 1}","{'layer': '350mb', 'rh': 0}","{'layer': '300mb', 'rh': -2}","{'layer': '250mb', 'rh': -3}","{'layer': '200mb', 'rh': 3}"


In [13]:
df_rh_profile = pd.DataFrame()
for i in range(len(df_rh.columns)): 

    #aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(df_rh[i].apply(pd.Series)["layer"][0]) 

    #hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(df_rh[i].apply(pd.Series)["rh"] )

    #usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df_rh_profile.insert(i, nombre, valores)

In [43]:
df_rh_profile

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb
0,14,12,7,5,5,2,-1,-2,-3,-3,-3,0,-1,2,2,-1
1,12,8,6,6,3,-1,-3,-2,-3,-3,-3,-2,0,2,0,-1
2,14,8,5,6,6,1,-3,-2,-3,-3,-4,-1,1,0,1,-2
3,14,10,8,6,4,3,-2,-2,-2,-3,-3,2,-1,1,0,0
4,15,12,10,6,5,2,-2,-2,-2,-2,-1,1,0,-2,-3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,2,2,2,-1,1,4,7,7,3,5,5,3,5,13,7,0
316,2,4,6,9,10,9,9,9,12,11,4,3,9,7,3,1
317,3,5,7,3,1,-1,-2,-1,11,11,11,1,-3,-2,3,1
318,4,4,6,8,8,7,6,-1,-3,-4,0,-2,-2,-3,-1,-1


### wind_profile

In [15]:
df_clima["wind_profile"].dtypes

dtype('O')

In [16]:
df_clima['wind_profile']= df_clima['wind_profile'].apply(ast.literal_eval)

In [17]:
df_wind = df_clima["wind_profile"].apply(pd.Series)
df_wind.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'direction': 95, 'speed': 4}","{'layer': '900mb', 'direction': 95, 'speed': 4}","{'layer': '850mb', 'direction': 90, 'speed': 5}","{'layer': '800mb', 'direction': 80, 'speed': 5}","{'layer': '750mb', 'direction': 65, 'speed': 4}","{'layer': '700mb', 'direction': 65, 'speed': 4}","{'layer': '650mb', 'direction': 65, 'speed': 3}","{'layer': '600mb', 'direction': 60, 'speed': 3}","{'layer': '550mb', 'direction': 80, 'speed': 2}","{'layer': '500mb', 'direction': 165, 'speed': 2}","{'layer': '450mb', 'direction': 195, 'speed': 3}","{'layer': '400mb', 'direction': 200, 'speed': 3}","{'layer': '350mb', 'direction': 235, 'speed': 5}","{'layer': '300mb', 'direction': 260, 'speed': 6}","{'layer': '250mb', 'direction': 260, 'speed': 6}","{'layer': '200mb', 'direction': 250, 'speed': 6}"
1,"{'layer': '950mb', 'direction': 90, 'speed': 4}","{'layer': '900mb', 'direction': 95, 'speed': 5}","{'layer': '850mb', 'direction': 85, 'speed': 5}","{'layer': '800mb', 'direction': 75, 'speed': 5}","{'layer': '750mb', 'direction': 65, 'speed': 4}","{'layer': '700mb', 'direction': 60, 'speed': 4}","{'layer': '650mb', 'direction': 55, 'speed': 3}","{'layer': '600mb', 'direction': 45, 'speed': 3}","{'layer': '550mb', 'direction': 60, 'speed': 2}","{'layer': '500mb', 'direction': 175, 'speed': 2}","{'layer': '450mb', 'direction': 200, 'speed': 3}","{'layer': '400mb', 'direction': 210, 'speed': 3}","{'layer': '350mb', 'direction': 235, 'speed': 5}","{'layer': '300mb', 'direction': 255, 'speed': 6}","{'layer': '250mb', 'direction': 245, 'speed': 6}","{'layer': '200mb', 'direction': 245, 'speed': 6}"
2,"{'layer': '950mb', 'direction': 90, 'speed': 4}","{'layer': '900mb', 'direction': 85, 'speed': 4}","{'layer': '850mb', 'direction': 75, 'speed': 4}","{'layer': '800mb', 'direction': 70, 'speed': 4}","{'layer': '750mb', 'direction': 60, 'speed': 4}","{'layer': '700mb', 'direction': 65, 'speed': 3}","{'layer': '650mb', 'direction': 60, 'speed': 3}","{'layer': '600mb', 'direction': 55, 'speed': 2}","{'layer': '550mb', 'direction': 100, 'speed': 2}","{'layer': '500mb', 'direction': 160, 'speed': 2}","{'layer': '450mb', 'direction': 200, 'speed': 3}","{'layer': '400mb', 'direction': 215, 'speed': 4}","{'layer': '350mb', 'direction': 245, 'speed': 6}","{'layer': '300mb', 'direction': 255, 'speed': 6}","{'layer': '250mb', 'direction': 220, 'speed': 6}","{'layer': '200mb', 'direction': 230, 'speed': 6}"
3,"{'layer': '950mb', 'direction': 95, 'speed': 4}","{'layer': '900mb', 'direction': 95, 'speed': 4}","{'layer': '850mb', 'direction': 85, 'speed': 4}","{'layer': '800mb', 'direction': 70, 'speed': 4}","{'layer': '750mb', 'direction': 65, 'speed': 4}","{'layer': '700mb', 'direction': 65, 'speed': 3}","{'layer': '650mb', 'direction': 60, 'speed': 3}","{'layer': '600mb', 'direction': 40, 'speed': 3}","{'layer': '550mb', 'direction': 45, 'speed': 2}","{'layer': '500mb', 'direction': 205, 'speed': 2}","{'layer': '450mb', 'direction': 210, 'speed': 3}","{'layer': '400mb', 'direction': 225, 'speed': 4}","{'layer': '350mb', 'direction': 255, 'speed': 5}","{'layer': '300mb', 'direction': 240, 'speed': 5}","{'layer': '250mb', 'direction': 190, 'speed': 5}","{'layer': '200mb', 'direction': 190, 'speed': 5}"
4,"{'layer': '950mb', 'direction': 110, 'speed': 4}","{'layer': '900mb', 'direction': 95, 'speed': 4}","{'layer': '850mb', 'direction': 90, 'speed': 4}","{'layer': '800mb', 'direction': 80, 'speed': 4}","{'layer': '750mb', 'direction': 70, 'speed': 4}","{'layer': '700mb', 'direction': 65, 'speed': 3}","{'layer': '650mb', 'direction': 70, 'speed': 3}","{'layer': '600mb', 'direction': 50, 'speed': 3}","{'layer': '550mb', 'direction': 45, 'speed': 2}","{'layer': '500mb', 'direction': 290, 'speed': 2}","{'layer': '450mb', 'direction': 230, 'speed': 3}","{'layer': '400mb', 'direction': 240, 'speed': 3}","{'layer': '350mb', 'direction': 260, 'speed': 5}","{'layer': '300mb', 'direction': 200, 'speed': 4}","{'layer': '250m

In [18]:
df_wind_direction_profile = pd.DataFrame()
for i in range(len(df_wind.columns)): 

    #aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "wind_direction_" + str(df_wind[i].apply(pd.Series)["layer"][0]) 

    #hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(df_wind[i].apply(pd.Series)["direction"])

    #usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df_wind_direction_profile.insert(i, nombre, valores)

In [19]:
df_wind_direction_profile.head()

,wind_direction_950mb,wind_direction_900mb,wind_direction_850mb,wind_direction_800mb,wind_direction_750mb,wind_direction_700mb,wind_direction_650mb,wind_direction_600mb,wind_direction_550mb,wind_direction_500mb,wind_direction_450mb,wind_direction_400mb,wind_direction_350mb,wind_direction_300mb,wind_direction_250mb,wind_direction_200mb
0,95,95,90,80,65,65,65,60,80,165,195,200,235,260,260,250
1,90,95,85,75,65,60,55,45,60,175,200,210,235,255,245,245
2,90,85,75,70,60,65,60,55,100,160,200,215,245,255,220,230
3,95,95,85,70,65,65,60,40,45,205,210,225,255,240,190,190
4,110,95,90,80,70,65,70,50,45,290,230,240,260,200,155,165


In [20]:
df_wind_speed_profile = pd.DataFrame()
for i in range(len(df_wind.columns)): 

    #aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "wind_speed_" + str(df_wind[i].apply(pd.Series)["layer"][0]) 

    #hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(df_wind[i].apply(pd.Series)["speed"])

    #usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df_wind_speed_profile.insert(i, nombre, valores)

In [21]:
df_wind_speed_profile.head()

,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,wind_speed_450mb,wind_speed_400mb,wind_speed_350mb,wind_speed_300mb,wind_speed_250mb,wind_speed_200mb
0,4,4,5,5,4,4,3,3,2,2,3,3,5,6,6,6
1,4,5,5,5,4,4,3,3,2,2,3,3,5,6,6,6
2,4,4,4,4,4,3,3,2,2,2,3,4,6,6,6,6
3,4,4,4,4,4,3,3,3,2,2,3,4,5,5,5,5
4,4,4,4,4,4,3,3,3,2,2,3,3,5,4,6,6


In [22]:
df_rh_profile.shape

(320, 16)

In [23]:
df_wind_direction_profile.shape

(320, 16)

In [24]:
df_wind_speed_profile.shape

(320, 16)

### Concatenación de los df rh y wind

In [25]:
df_profile = pd.concat([df_wind_direction_profile, df_wind_speed_profile, df_rh_profile], axis=1)

In [26]:
df_profile.shape

(320, 48)

In [27]:
df_profile.head()

,wind_direction_950mb,wind_direction_900mb,wind_direction_850mb,wind_direction_800mb,wind_direction_750mb,wind_direction_700mb,wind_direction_650mb,wind_direction_600mb,wind_direction_550mb,wind_direction_500mb,wind_direction_450mb,wind_direction_400mb,wind_direction_350mb,wind_direction_300mb,wind_direction_250mb,wind_direction_200mb,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,wind_speed_450mb,wind_speed_400mb,wind_speed_350mb,wind_speed_300mb,wind_speed_250mb,wind_speed_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb
0,95,95,90,80,65,65,65,60,80,165,195,200,235,260,260,250,4,4,5,5,4,4,3,3,2,2,3,3,5,6,6,6,14,12,7,5,5,2,-1,-2,-3,-3,-3,0,-1,2,2,-1
1,90,95,85,75,65,60,55,45,60,175,200,210,235,255,245,245,4,5,5,5,4,4,3,3,2,2,3,3,5,6,6,6,12,8,6,6,3,-1,-3,-2,-3,-3,-3,-2,0,2,0,-1
2,90,85,75,70,60,65,60,55,100,160,200,215,245,255,220,230,4,4,4,4,4,3,3,2,2,2,3,4,6,6,6,6,14,8,5,6,6,1,-3,-2,-3,-3,-4,-1,1,0,1,-2
3,95,95,85,70,65,65,60,40,45,205,210,225,255,240,190,190,4,4,4,4,4,3,3,3,2,2,3,4,5,5,5,5,14,10,8,6,4,3,-2,-2,-2,-3,-3,2,-1,1,0,0
4,110,95,90,80,70,65,70,50,45,290,230,240,260,200,155,165,4,4,4,4,4,3,3,3,2,2,3,3,5,4,6,6,15,12,10,6,5,2,-2,-2,-2,-2,-1,1,0,-2,-3,3


### Eliminar las columnas rh_profile y wind_profile del df_clima

In [28]:
df_clima = df_clima.drop(["rh_profile", "wind_profile"], axis=1)

In [29]:
df_clima.shape

(320, 16)

In [30]:
df_clima.head(2)

,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,0,3,1,-9999,-9999,-9999,26,-1,12,1011,none,1,0,95,3,papua new guinea
1,1,6,1,-9999,-9999,-9999,26,-1,12,1013,none,1,0,90,3,papua new guinea


### Merge de df_clima y df_profile

In [31]:
df_clima_completo = df_clima.join(df_profile, how="inner", on="index")

In [32]:
df_clima_completo.shape

(320, 64)

In [33]:
df_clima_completo.head()

,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais,wind_direction_950mb,wind_direction_900mb,wind_direction_850mb,wind_direction_800mb,wind_direction_750mb,wind_direction_700mb,wind_direction_650mb,wind_direction_600mb,wind_direction_550mb,wind_direction_500mb,wind_direction_450mb,wind_direction_400mb,wind_direction_350mb,wind_direction_300mb,wind_direction_250mb,wind_direction_200mb,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,wind_speed_450mb,wind_speed_400mb,wind_speed_350mb,wind_speed_300mb,wind_speed_250mb,wind_speed_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb
0,0,3,1,-9999,-9999,-9999,26,-1,12,1011,none,1,0,95,3,papua new guinea,95,95,90,80,65,65,65,60,80,165,195,200,235,260,260,250,4,4,5,5,4,4,3,3,2,2,3,3,5,6,6,6,14,12,7,5,5,2,-1,-2,-3,-3,-3,0,-1,2,2,-1
1,1,6,1,-9999,-9999,-9999,26,-1,12,1013,none,1,0,90,3,papua new guinea,90,95,85,75,65,60,55,45,60,175,200,210,235,255,245,245,4,5,5,5,4,4,3,3,2,2,3,3,5,6,6,6,12,8,6,6,3,-1,-3,-2,-3,-3,-3,-2,0,2,0,-1
2,2,9,1,-9999,-9999,-9999,26,2,11,1012,none,1,0,90,3,papua new guinea,90,85,75,70,60,65,60,55,100,160,200,215,245,255,220,230,4,4,4,4,4,3,3,2,2,2,3,4,6,6,6,6,14,8,5,6,6,1,-3,-2,-3,-3,-4,-1,1,0,1,-2
3,3,12,1,-9999,-9999,-9999,26,-1,12,1011,none,1,0,100,3,papua new guinea,95,95,85,70,65,65,60,40,45,205,210,225,255,240,190,190,4,4,4,4,4,3,3,3,2,2,3,4,5,5,5,5,14,10,8,6,4,3,-2,-2,-2,-3,-3,2,-1,1,0,0
4,4,15,1,-9999,-9999,-9999,25,-1,11,1012,none,1,0,105,3,papua new guinea,110,95,90,80,70,65,70,50,45,290,230,240,260,200,155,165,4,4,4,4,4,3,3,3,2,2,3,3,5,4,6,6,15,12,10,6,5,2,-2,-2,-2,-2,-1,1,0,-2,-3,3


### Group by por paises 

In [34]:
df_grupo = df_clima_completo.groupby("pais")["temp2m"].mean().reset_index()

In [35]:
df_grupo

,pais,temp2m
0,australia,25.890625
1,new zealand,15.062500
2,papua new guinea,25.890625
3,south africa,23.421875
4,usa,12.718750


In [36]:
df_grupo = df_clima_completo.groupby("pais")[df_clima_completo.columns].mean().reset_index()

/tmp/ipykernel_1321/2829929470.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_grupo = df_clima_completo.groupby("pais")[df_clima_completo.columns].mean().reset_index()


In [37]:
df_grupo

,pais,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,wind_direction_950mb,wind_direction_900mb,wind_direction_850mb,wind_direction_800mb,wind_direction_750mb,wind_direction_700mb,wind_direction_650mb,wind_direction_600mb,wind_direction_550mb,wind_direction_500mb,wind_direction_450mb,wind_direction_400mb,wind_direction_350mb,wind_direction_300mb,wind_direction_250mb,wind_direction_200mb,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,wind_speed_450mb,wind_speed_400mb,wind_speed_350mb,wind_speed_300mb,wind_speed_250mb,wind_speed_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb
0,australia,223.5,97.5,3.703125,-9999.0,-9999.0,-9999.0,25.890625,-3.750000,10.906250,1014.906250,2.859375,0.0,122.968750,3.296875,121.406250,115.703125,129.609375,135.546875,136.250000,133.828125,136.484375,132.421875,133.906250,129.453125,121.015625,120.546875,123.203125,138.203125,151.875000,161.015625,3.671875,3.640625,3.500000,3.406250,3.375000,3.390625,3.312500,3.421875,3.703125,3.843750,4.000000,4.031250,4.250000,4.593750,5.093750,5.671875,13.609375,11.234375,8.406250,6.796875,5.671875,4.187500,3.515625,2.796875,2.593750,2.375000,2.171875,2.078125,1.546875,1.640625,4.125000,6.984375
1,new zealand,95.5,97.5,6.734375,-9999.0,-9999.0,-9999.0,15.062500,9.093750,10.859375,1015.500000,3.203125,0.0,149.453125,3.359375,157.890625,159.921875,178.359375,191.875000,194.765625,212.500000,218.984375,223.125000,216.328125,219.843750,233.515625,241.796875,254.843750,264.609375,276.484375,288.593750,3.687500,3.625000,3.500000,3.500000,3.484375,3.546875,3.609375,3.765625,3.984375,4.140625,4.484375,4.796875,5.093750,5.406250,6.406250,7.500000,13.359375,12.906250,7.921875,5.296875,3.265625,2.671875,1.875000,1.703125,3.140625,4.640625,4.906250,5.109375,5.031250,4.046875,1.421875,-1.812500
2,papua new guinea,31.5,97.5,3.546875,-9999.0,-9999.0,-9999.0,25.890625,-0.671875,11.500000,1010.265625,2.109375,0.0,84.375000,3.156250,82.812500,82.031250,82.109375,80.390625,77.343750,79.843750,145.234375,196.953125,240.781250,260.859375,256.796875,243.281250,239.140625,220.859375,190.468750,221.562500,3.828125,4.250000,4.296875,4.109375,3.406250,2.828125,2.437500,2.734375,3.218750,3.703125,4.062500,4.218750,4.265625,4.140625,4.687500,5.078125,13.578125,11.531250,9.343750,5.812500,2.531250,0.343750,1.296875,0.031250,-1.812500,-1.906250,0.296875,1.750000,2.218750,2.218750,2.125000,1.171875
3,south africa,159.5,97.5,7.031250,-9999.0,-9999.0,-9999.0,23.421875,2.562500,10.156250,1019.484375,1.781250,0.0,152.500000,2.296875,147.656250,142.812500,159.062500,181.484375,196.250000,202.968750,203.125000,200.468750,211.406250,215.000000,216.875000,219.218750,216.953125,212.812500,209.687500,208.437500,2.359375,2.265625,2.218750,2.140625,2.156250,2.234375,2.484375,2.781250,3.062500,3.375000,3.843750,4.140625,4.578125,5.093750,5.359375,5.968750,12.328125,8.703125,6.546875,7.421875,8.812500,9.953125,9.984375,8.296875,5.625000,4.562500,4.687500,4.953125,5.656250,6.312500,7.593750,9.390625
4,usa,287.5,97.5,3.562500,-9999.0,-9999.0,-9999.0,12.718750,13.265625,4.812500,1022.968750,0.000000,0.0,203.750000,2.093750,222.812500,230.468750,276.875000,279.921875,271.718750,267.187500,261.562500,261.015625,267.656250,263.671875,271.640625,260.234375,240.468750,235.390625,222.500000,248.828125,2.343750,2.515625,2.953125,3.296875,3.531250,3.937500,4.296875,4.484375,4.609375,4.734375,4.968750,5.046875,5.078125,5.421875,5.640625,5.484375,4.187500,4.000000,3.890625,3.640625,3.546875,3.265625,2.453125,1.703125,2.046875,3.281250,4.921875,5.046875,6.078125,6.656250,5.937500,5.734375


### Unión df_ataques y df_grupo

In [38]:
df_completo = df_ataques.merge(df_grupo, how="inner", right_on="pais", left_on="country")

In [39]:
df_completo.shape

(4335, 84)

In [40]:
df_completo.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM,pais,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,wind_direction_950mb,wind_direction_900mb,wind_direction_850mb,wind_direction_800mb,wind_direction_750mb,wind_direction_700mb,wind_direction_650mb,wind_direction_600mb,wind_direction_550mb,wind_direction_500mb,wind_direction_450mb,wind_direction_400mb,wind_direction_350mb,wind_direction_300mb,wind_direction_250mb,wind_direction_200mb,wind_speed_950mb,wind_speed_900mb,wind_speed_850mb,wind_speed_800mb,wind_speed_750mb,wind_speed_700mb,wind_speed_650mb,wind_speed_600mb,wind_speed_550mb,wind_speed_500mb,wind_speed_450mb,wind_speed_400mb,wind_speed_350mb,wind_speed_300mb,wind_speed_250mb,wind_speed_200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327,usa,287.5,97.5,3.5625,-9999.0,-9999.0,-9999.0,12.71875,13.265625,4.8125,1022.96875,0.0,0.0,203.75,2.09375,222.8125,230.46875,276.875,279.921875,271.71875,267.1875,261.5625,261.015625,267.65625,263.671875,271.640625,260.234375,240.46875,235.390625,222.5,248.828125,2.34375,2.515625,2.953125,3.296875,3.53125,3.9375,4.296875,4.484375,4.609375,4.734375,4.96875,5.046875,5.078125,5.421875,5.640625,5.484375,4.1875,4.0,3.890625,3.640625,3.546875,3.265625,2.453125,1.703125,2.046875,3.28125,4.921875,5.046875,6.078125,6.65625,5.9375,5.734375
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860,usa,287.5,97.5,3.5625,-9999.0,-9999.0,-9999.0,12.71875,13.265625,4.8125,1022.96875,0.0,0.0,203.75,2.09375,222.8125,230.46875,276.875,279.921875,271.71875,267.1875,261.5625,261.015625,267.65625,263.671875,271.640625,260.234375,240.46875,235.390625,222.5,248.828125,2.34375,2.515625,2.953125,3.296875,3.53125,3.9375,4.296875,4.484375,4.609375,4.734375,4.96875,5.046875,5.078125,5.421875,5.640625,5.484375,4.1875,4.0,3.890625,3.640625,3.546875,3.265625,2.453125,1.703125,2.046875,3.28125,4.921875,5.046875,6.078125,6.65625,5.9375,5.734375


In [41]:
df_completo.to_csv("etl2.csv")